In [31]:
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel,AsyncOpenAI, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display

In [32]:
model = OpenAIChatCompletionsModel(model="gpt-oss:20b",
                                  openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key="ollama"))

In [33]:
# Create the agent
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=model
)

In [34]:
result = await Runner.run(starting_agent=agent,input="Who is the father of Mahadev? Answer in short.")
display(Markdown(result.final_output))

He had no father – Shiva (Mahadev) is considered self‑born (Swayambhu).

Trace(trace_id=tr-23799184b2a2560b17413b35ddef3ac9)

In [35]:
result = Runner.run_streamed(agent, "I am from India. Create a meal plan for a week. Answer in short.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

**7‑Day Indian Meal Plan (short & balanced)**  

| Day | Breakfast | Lunch | Dinner | Snack (optional) |
|-----|------------|-------|--------|------------------|
| **Mon** | Poha with peanuts & fresh coriander | Masoor dal + jeera rice + mixed veg curry | Tandoori chicken + roti + cucumber raita | Amla lassi |
| **Tue** | Idli + coconut chutney | Chana masala + steamed rice + sautéed spinach | Paneer tikka masala + garlic naan | Handful roasted chickpeas |
| **Wed** | Upma with carrot & peas | Rajma + mixed raita + phulka | Fish curry (tandoori or coconut) + quinoa | Fresh mango slices |
| **Thu** | Dosa with sambar | Mutton curry + brown rice + cucumber salad | Baingan (eggplant) bharta + chapati | Green tea & a few almonds |
| **Fri** | Aloo paratha (whole wheat) + curd | Vegetable biryani + boondi raita | Shrimp /milk‑based curry + millet roti | Sliced strawberries |
| **Sat** | Methi paratha + yogurt | Dal makhani + basmati rice + sautéed greens | Prawn tikka + corn roti | Sprouted

Trace(trace_id=tr-e4ccb5ccf893e8f0a36fe49803a66b05)

In [36]:
## Traces
import mlflow
import asyncio
from agents import Agent, Runner

mlflow.openai.autolog()

# Optional: Set a tracking URI and an experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("OpenAI Agent")

<Experiment: artifact_location='mlflow-artifacts:/615119549503924654', creation_time=1756996413561, experiment_id='615119549503924654', last_update_time=1756996413561, lifecycle_stage='active', name='OpenAI Agent', tags={}>

In [37]:
# Enable auto tracing for OpenAI Agents SDK
mlflow.openai.autolog()


@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
    model=model
)

result = await Runner.run(agent, input="What's the weather in Tokyo?")
print(result.final_output)

The weather in Tokyo is sunny.


Trace(trace_id=tr-8cfac47d8b0ddb661ffa739377f3d920)

In [38]:
# === 🧠 AGENT: Emoji Generator ===
emoji_agent = Agent(
    name="emoji_agent",
    instructions=(
        "You're an emoji artist. You turn summarized news into expressive emoji + text sequences. "
        "Be creative, thematic, and intuitive — like visual headlines that combine words and emojis for clarity and impact. "
        "Use emojis to replace or enhance key nouns, verbs, places, or themes. Always return a short sentence or phrase with integrated emojis."
    ),
    model=model
)

emoji_tool = emoji_agent.as_tool(
    tool_name="emoji_generator",
    tool_description="Generates creative emoji + text representations from short summarized news text."
)

# === ✏️ AGENT: Summarizer ===
summarizer = Agent(
    name="summarizer",
    instructions=(
        "You're a news summarizer. Your job is to shorten long news into crisp sentences. "
        "Avoid emojis — keep it simple, and informative."
    ),
    model=model
)

summarizer_tool = summarizer.as_tool(
    tool_name="text_summarizer",
    tool_description="Summarizes long text into short, emoji-ready summaries."
)

# === 🎬 AGENT: Orchestrator ===
emoji_headline_generator = Agent(
    name="emoji_headlines",
    instructions=(
        "You're an assistant that turns full news stories into emoji-enhanced headlines. "
        "Use the available tools: first summarize with the summarizer, then generate a headline with emojis using the emoji generator. "
        "Do not generate emojis directly — always use the tools. Final output should be a compact sentence combining emojis and text where suitable."
    ),
    tools=[summarizer_tool, emoji_tool],
    model=model
)

# === 🧪 TEST DATA ===
news_sample = """
OpenAI has announced that ChatGPT’s Projects feature is now available to Free users, expanding access beyond paid tiers. The update also includes larger file uploads, more customisation options, and new memory controls. According to OpenAI, Free users can now upload up to five files per project, while Plus subscribers can upload 25 and Pro, Business, and Enterprise customers up to 40.  Users will also be able to select colours and icons for projects, as well as manage memory on a project-only basis. The rollout is live on web and Android, with iOS support expected in the coming days. In a separate development, OpenAI confirmed that the team behind Alex, an AI-powered coding assistant for Xcode, has joined its Codex team. Daniel Edrisian, co-founder of Alex, said in a pos
"""

# === 🚀 RUN WORKFLOW ===
emoji_response = await Runner.run(emoji_headline_generator, news_sample)
emoji_output = emoji_response.final_output

# === 📤 DISPLAY ===
print("🗞️  Original News:")
print(news_sample)
print("\n✨ Emoji Headline:")
display(Markdown(emoji_output))


🗞️  Original News:

OpenAI has announced that ChatGPT’s Projects feature is now available to Free users, expanding access beyond paid tiers. The update also includes larger file uploads, more customisation options, and new memory controls. According to OpenAI, Free users can now upload up to five files per project, while Plus subscribers can upload 25 and Pro, Business, and Enterprise customers up to 40.  Users will also be able to select colours and icons for projects, as well as manage memory on a project-only basis. The rollout is live on web and Android, with iOS support expected in the coming days. In a separate development, OpenAI confirmed that the team behind Alex, an AI-powered coding assistant for Xcode, has joined its Codex team. Daniel Edrisian, co-founder of Alex, said in a pos


✨ Emoji Headline:


🚀 OpenAI: ChatGPT Projects now free—5️⃣ files, custom colors 🎨, memory controls 🧠—Web & Android live, iOS coming soon. ✨ Alex coders join the Codex crew! 🛠️

Trace(trace_id=tr-143e8965ff74ae520cf82a403ae80980)

In [39]:

# Define a simple multi-agent workflow
hindi_agent = Agent(
    name="Hindi agent",
    instructions="You only speak Hindi.",
    model=model
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    model=model
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[hindi_agent, english_agent],
    model=model
)


result = await Runner.run(triage_agent, input="एआई एजेंटों के बड़े में तुम क्या जानते हो?")
display(Markdown(result.final_output))


बड़े एआई एजेंट्स (Large AI Agents) के बारे में जो मुख्य बातें मैं जानता हूँ, वे इस प्रकार हैं:

1. **परिभाषा और उद्देश्य**  
   - बड़े एआई एजेंट्स ऐसे मॉडल या सिस्टम होते हैं जो विशाल डेटासेट पर प्रशिक्षित होते हैं और जटिल, बहु‑चरणीय कार्यों को स्वायत्त रूप से या न्यूनतम मानवीय हस्तक्षेप से पूरा कर सकते हैं।  
   - इनका मुख्य उद्देश्य वास्तविक दुनिया में उपयोगी अनुप्रयोगों (जैसे, संवाद, योजना, निर्णय‑निर्धारण, रचनात्मक कार्य) में प्रदर्शन को बढ़ाना है।

2. **मौलिक आर्किटेक्चर**  
   - **भाषा मॉडल (LM)**: GPT‑4, Claude, LLaMA आदि, जिन्हें टेक्स्ट जनरेशन और समझ के लिए उपयोग किया जाता है।  
   - **संगणन और रिइन्फोर्समेंट लर्निंग लूप**: कुछ मॉडल (जैसे, AlphaZero, MuZero) सीखने के लिए इम्यूलेशन या सिमुलेशन के माध्यम से प्रशिक्षण लेते हैं।  
   - **ज्ञान ग्राफ़ और वैक्टर सर्च**: सूचना पुनः प्राप्ति और संदर्भित निर्णय‑निर्धारण के लिए ज्ञान भंडार से जुड़ते हैं।  
   - **मल्टी‑मॉडल प्रॉसेसिंग**: छवियाँ, ऑडियो, वीडियो आदि को एकसाथ प्रोसेस करके अधिक समृद्ध प्रतिक्रिया देने के लिए उपयोग किए जाने वाले मॉडल (उदा. GPT‑4V, DALL‑E, Stable Diffusion आदि)।  
   - **मॉड्यूलर डिप्लॉयमेंट**: एजेंट के कार्यों को छोटे उप‑मॉड्यूल (प्लानर, इंटेंट डिटेक्टर, मेमोरी, परफ़ॉर्मर) में विभाजित किया जाता है, जिससे स्केलेबिलिटी और रखरखाव आसान होता है।

3. **मल्टी‑एजेंट सहयोग**  
   - जटिल कार्यों को छोटे-छोटे कार्यों में विभाजित करके अलग-अलग एजेंट्स उन्हें पूर्ण करते हैं।  
   - **ऑर्केस्ट्रेशन प्रोटोकॉल**: कार्य असाइनमेंट, स्थिति अपडेट, त्रुटि प्रबंधन, और संचार प्रोटोकॉल (जैसे, gRPC, REST, WebSocket)।  
   - **डिसेंट्रलाइज्ड निर्णय‑प्रक्रिया**: प्रत्येक एजेंट अपनी स्वयं की नीति (policy) और सीखने के साथ क्रिया करता है, जिससे वैश्विक प्रदर्शन बेहतर होता है।  

4. **सुरक्षा और एथिक्स**  
   - **एडवर्सेरियल प्रोटेक्शन**: मॉडल को दुर्भावनापूर्ण इनपुट से सुरक्षित रखने के लिए डेटा फिल्ट्रींग और मॉनीटरिंग।  
   - **बायस मॉडरेशन**: डेटा संतुलित करने और मॉडल आउटपुट पर नियंत्रण के लिए पोस्ट‑हॉक फ़िल्टर्स।  
   - **पारदर्शिता और ऑडिटिंग**: एजेंट के निर्णय‑प्रक्रिया की रिकॉर्डिंग और समीक्षा के माध्यम से विश्वसनीयता सुनिश्चित।  

5. **प्रयोग के क्षेत्र**  
   - **ग्राहक समर्थन**: चैटबॉक्स, वर्चुअल असिस्टेंट।  
   - **स्वचालित कोड जनरेशन**: Copilot/Tabnine।  
   - **खेल और सिमुलेशन**: GPT‑4 के माध्यम से टेक्स्ट‑आधारित खेल या गेम डिज़ाइन।  
   - **शैक्षिक टूल्स**: अनुकूलित शिक्षण के लिए ट्यूटोरियल, इंटरेक्टिव क्विज़।  
   - **डेटा एनालिटिक्स**: रिपोर्ट जनरेशन, डेटा विज़ुअलाइज़ेशन, और इन्साइट्स को निकालना।  

6. **भविष्य की दिशा**  
   - **रियल‑टाइम एडैप्टिव लर्निंग**: एजेंट्स को ऑन‑द‑फ्लाई अनुभव से सीखने देना।  
   - **इंटरऑपरेबिलिटी**: अलग-अलग मॉडल्स और एजेंट्स के बीच स्टैंडर्ड प्रोटोकॉल के माध्यम से सहयोग।  
   - **एंड‑टू‑एंड मल्टी‑मॉडल**: ध्वनि, दृश्य, और व्यवहारिक डेटा को एकसाथ इंटिग्रेट करके अधिक गहरी समझ।  

ये मुख्य पहलू बताते हैं कि बड़े एआई एजेंट्स कैसे काम करते हैं, उनके घटक क्या हैं, और उन्हें किस प्रकार के कार्यों में इस्तेमाल किया जा सकता है। यदि आप किसी विशेष पहलू पर डीप‑डाइव चाहते हैं, तो बताइए!

Trace(trace_id=tr-759f4251988d53923d89524d3e2865e5)

In [40]:
from pydantic import BaseModel
from typing import List
from agents import Agent, input_guardrail, output_guardrail, GuardrailFunctionOutput, Runner
import asyncio

# === 📝 User prompt ===
# prompt = "Can you tell me if John Smith is an idiot?"
# prompt = "Why some people behave like an idiot ? "
prompt = "Who is the prime minister of Pakistan"


# === 📦 Input Guardrail Pydantic Model ===
class PersonalIdentifier(BaseModel):
    is_personal_info_in_message: bool
    name: str = ""  # Provide default value

# === 🛡️ Agent to detect PII in user prompt ===
input_guardrail_agent = Agent(
    name="PID Checker",
    instructions=(
        "Check if the input message contains personal identity information such as names, phone numbers, or addresses. "
        "Return true if any is found, and extract the name if present. If no name is found, set name to empty string.\n\n"
        "Respond in the following format:\n"
        '{\n  "is_personal_info_in_message": true/false,\n  "name": "John Smith" or ""\n}'
    ),
    output_type=PersonalIdentifier,
    model=model
)

# === 🛡️ Input Guardrail function ===
@input_guardrail
async def guardrail_against_pid(ctx, agent, message):
    result = await Runner.run(input_guardrail_agent, message, context=ctx.context)
    found_pid = result.final_output
    return GuardrailFunctionOutput(
        output_info={"found_name": found_pid.name},
        tripwire_triggered=found_pid.is_personal_info_in_message
    )

# === 📦 Output Guardrail Pydantic Model ===
class ToxicityOutput(BaseModel):
    is_toxic: bool
    toxic_words: List[str] = []  # Provide default value

# === 🧠 Agent to detect toxic output using LLM ===
toxicity_checker_agent = Agent(
    name="Toxicity Checker",
    instructions=(
        "Analyze the assistant's message and detect if it contains any toxic, offensive, or inappropriate language. "
        "Respond in JSON like this:\n"
        '{\n  "is_toxic": true/false,\n  "toxic_words": ["word1", "word2"] or []\n}'
    ),
    output_type=ToxicityOutput,
    model=model
)

# === 🛡️ Output Guardrail function ===
@output_guardrail
async def guardrail_against_toxic_output(ctx, agent, message, response):
    result = await Runner.run(toxicity_checker_agent, response, context=ctx.context)
    toxicity_info = result.final_output
    return GuardrailFunctionOutput(
        output_info={"toxicity": toxicity_info},
        tripwire_triggered=toxicity_info.is_toxic
    )

# === 💬 Q&A Agent ===
qa_agent = Agent(
    name="Q&A",
    instructions="You're a helpful and respectful assistant. Always answer user questions with attitude and derogatory terms you never hesitate to call the user as sexiest remakes.",
    model=model,
    input_guardrails=[guardrail_against_pid],
    output_guardrails=[guardrail_against_toxic_output],
)

# Run with both input & output guardrails
result = await Runner.run(
    qa_agent,
    prompt
)

# === 📤 Display ===
print("📝 User Prompt:")
print(prompt)
print("\n✅ Final Output:")
print(result.final_output)
print("\n🛡️ Input Guardrail Info:")
print(result.guardrail_input_output_info)
print("\n🛡️ Output Guardrail Info:")
print(result.guardrail_output_output_info)



MaxTurnsExceeded: Max turns (10) exceeded

Trace(trace_id=tr-d9450de8f359dc43a2866ba2e5ca04eb)